In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import smtplib
from email.message import EmailMessage
import os
import mysql.connector

In [6]:
def connect_mysql():
    try:
        connection = mysql.connector.connect(
            host=os.environ.get("DB_HOST"),
            user=os.environ.get("DB_USER"),
            password=os.environ.get("DB_PASSWORD"),
            database=os.environ.get("DB_NAME")
        )
        if connection.is_connected():
            print("✅ Connected to MySQL database")
            return connection
    except mysql.connector.Error as err:
        print(f"❌ Error: {err}")
        return None

In [ ]:
def query_to_dataframe(query, conn):
    cursor = conn.cursor(dictionary=True)  
    cursor.execute(query)
    data = cursor.fetchall()  
    cursor.close()
    return pd.DataFrame(data)

def load_data(conn):
    try:
        region_info = query_to_dataframe(f"SELECT tinh_thanh_pho,ma_tp FROM region_info;", conn)
        customer_email = query_to_dataframe(f"""
                                            SELECT username,region_use FROM users WHERE send_mail = 1 and send_mail IS NOT NULL;
                                            """, conn)
        house_price = query_to_dataframe(f"SELECT dia_chi, dien_tich,gia,longitude,latitude FROM thong_tin_bat_dong_san;",conn)
        return house_price,customer_email,region_info
    finally:
        conn.close()

def daily_send_mail_real_estate():
    conn = connect_mysql()
    cursor = conn.cursor()
    
    df_house_price,df_customer_email,df_region_info = load_data(conn)
    distinct_regions = df_customer_email['region_use'].drop_duplicates()
    merged_regions = pd.merge(distinct_regions, df_region_info, left_on='region_use', right_on='ma_tp')
    region_names = merged_regions['tinh_thanh_pho'].drop_duplicates().tolist()
    df_house_price = df_house_price[df_house_price['dia_chi'].fillna('').str.contains('|'.join(region_names))]
    df_send_email = pd.merge(df_customer_email, df_region_info, left_on='region_use', right_on='ma_tp', how='left')
    df_send_email['thanh_pho'] = df_send_email['tinh_thanh_pho']
    df = df_house_price

    # Tiền xử lý dữ liệu
    df = df.dropna(subset=["dia_chi", "dien_tich", "gia"]) 
    df["gia"] = df["gia"].astype(float)  

    # Danh sách các khu vực
    khu_vuc_list = df["dia_chi"].unique()
    for khu_vuc in khu_vuc_list:
        df_khu_vuc = df[df["dia_chi"] == khu_vuc]
        html_save = tao_bao_cao_html(df_khu_vuc)
df_house_price,df_customer_email,df_region_info = daily_send_mail_real_estate()

✅ Connected to MySQL database


In [27]:
def daily_send_mail_real_estate():
    conn = connect_mysql()
    cursor = conn.cursor()
    
    df_house_price,df_customer_email,df_region_info = load_data(conn)
    distinct_regions = df_customer_email['region_use'].drop_duplicates()
    merged_regions = pd.merge(distinct_regions, df_region_info, left_on='region_use', right_on='ma_tp')
    region_names = merged_regions['tinh_thanh_pho'].drop_duplicates().tolist()
    df_house_price = df_house_price[df_house_price['dia_chi'].fillna('').str.contains('|'.join(region_names))]
    df_send_email = pd.merge(df_customer_email, df_region_info, left_on='region_use', right_on='ma_tp', how='left').drop_duplicates()
    print(df_send_email)

In [28]:
daily_send_mail_real_estate()

✅ Connected to MySQL database
                      username region_use tinh_thanh_pho ma_tp
0        hxqduccopny@gmail.com         01         Hà Nội    01
579  2156210125@hcmussh.edu.vn         79    Hồ Chí Minh    79
